# Set up

In [1]:
!echo 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

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


In [ ]:
#!pip install -U langchain 

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "stage_1"

In [3]:
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from langchain.callbacks.tracers import LangChainTracer

In [5]:
from langchain_anthropic import ChatAnthropic

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

#model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)


# Utils

In [79]:
from typing import List

def initialize_agent_state(transcript: str, meeting_type: str, main_topic: str, context: List[str], max_revisions : int) -> AgentState:
	"""
	Initialize the agent state with the given parameters.

	Args:
	transcript (str): The meeting transcript.
	meeting_type (str): The type of the meeting.
	main_topic (str): The main topic of the meeting.
	context (List[str]): Additional context information.
	max_revisions (int): The maximume number of revison.
	Returns:
	AgentState: The initialized agent state.
	"""
	return AgentState(
		transcript=transcript,
		meeting_type=meeting_type,
		main_topic=main_topic,
		max_revisions=max_revisions,
		context=context,
		summary_plan="",
		critique_plan="",
		summary="",
		critique="",
		final_summary= "",
		revision_number=0,
		metadata={},
		action_items=[],
		decisions=[]
	)

In [9]:
def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

In [10]:
from langchain import PromptTemplate

def load_and_format_prompt(prompt_path, **kwargs):
	# Load the prompt template
	with open(prompt_path, 'r', encoding='utf-8') as file:
		template = file.read()
	
	# Create a PromptTemplate
	prompt = PromptTemplate(
		input_variables=list(kwargs.keys()),
		template=template
	)
	
	# Format the prompt with provided variables
	formatted_prompt = prompt.format(**kwargs)
	
	return formatted_prompt

In [ ]:
summary_draft = "This is a draft summary of the meeting..."
meeting_transcript = "Full transcript of the meeting goes here..."
summary_plan = "The plan for summarizing the meeting..."
load_and_format_prompt(file_path="./prompts/updated-plan-generator-prompt.md", )

# Agents

In [78]:
class AgentState(TypedDict):
	transcript: str 		# get from UI
	meeting_type: str 		# get from UI
	main_topic: str  		# get from UI
	max_revisions: int		# get from UI
	context: list[str] 		# get from database [later optinaly can be given by user]
	summary_plan: str		# populated by summary_plan_node
	critique_plan: str		# populated by critique_plan_node
	summary: str			# populated by summarizer_node
	critique: str			# populated by critic_node
	final_summary: str		# populated by finalization_node
	revision_number: int 	# incremented by critic_node if revision is needed
	metadata: dict			# collect additional information
	action_items: list[str] 
	decisions: list[str]

In [42]:
# Load prompts
SUMMARY_PLAN_PROMPT = load_markdown_to_str(file_path="../data/prompts/updated-plan-generator-prompt.md")
CRITIQUE_PLAN_PROMPT = load_markdown_to_str(file_path="../data/prompts/updated-critique-planner-prompt.md")
SUMMARIZER_PROMPT = load_markdown_to_str(file_path="../data/prompts/draft-generation-prompt.md")
CRITIC_PROMPT = load_markdown_to_str(file_path="../data/prompts/critique-agent-prompt.md")



In [40]:
# Load prompts with variables
meeting_type = "Sprint Planning áá"
main_topic = "Design the next 2 weeks of the sprint áá"
summary_draft = "aaa"
critique_plan = "ááá"
context = "áá"
transcript = "ááá"
SUMMARY_PLAN_PROMPT = load_and_format_prompt(prompt_path="../data/prompts/updated-plan-generator-prompt.md", meeting_tpye= meeting_type,main_topic= main_topic)
CRITIQUE_PLAN_PROMPT = load_and_format_prompt(prompt_path="../data/prompts/updated-critique-planner-prompt.md", meeting_type = meeting_type, main_topic= main_topic)
SUMMARIZER_PROMPT = load_and_format_prompt(prompt_path="../data/prompts/draft-generation-prompt.md",summary_plan = summary_plan, context = context, transcript = transcript)
CRITIC_PROMPT= load_and_format_prompt(prompt_path="../data/prompts/critique-agent-prompt.md",summary_draft = summary_draft, transcript= transcript, critique_plan= critique_plan)


TODO: Handle variables

In [44]:
def summary_plan_node(state: AgentState) -> AgentState:
    messages = [
        SystemMessage(content=SUMMARY_PLAN_PROMPT),
        HumanMessage(content=f"Meeting Type: {state['meeting_type']}\nMain Topic: {state['main_topic']}")
    ]
    response = model.invoke(messages)
    state["summary_plan"] = response.content
    return state

In [46]:
def critique_plan_node(state: AgentState) -> AgentState:
    messages = [
        SystemMessage(content=CRITIQUE_PLAN_PROMPT),
        HumanMessage(content=f"Meeting Type: {state['meeting_type']}\nMain Topic: {state['main_topic']}")
    ]
    response = model.invoke(messages)
    state["critique_plan"] = response.content
    return state

In [20]:
def summarizer_node(state: AgentState) -> AgentState:
	messages = [
		SystemMessage(content=SUMMARIZER_PROMPT),
		HumanMessage(content=f"""
		Meeting Type: {state['meeting_type']}
		Main Topic: {state['main_topic']}
		Summary Plan: {state['summary_plan']}
		Transcript: {state['transcript']}
		Previous Summary: {state['summary']}
		Revision Number: {state['revision_number']}
		""")
	]
	response = model.invoke(messages)
	state["summary"] = response.content
	
	# Extract action items and decisions (this is a simple implementation and might need refinement)
	state["action_items"] = [line for line in response.content.split('\n') if line.startswith("Action Item:")]
	state["decisions"] = [line for line in response.content.split('\n') if line.startswith("Decision:")]
	
	return state

In [21]:
def critic_node(state: AgentState) -> AgentState:
	messages = [
		SystemMessage(content=CRITIC_PROMPT),
		HumanMessage(content=f"""
		Meeting Type: {state['meeting_type']}
		Main Topic: {state['main_topic']}
		Critique Plan: {state['critique_plan']}
		Summary: {state['summary']}
		Transcript: {state['transcript']}
		""")
	]
	response = model.invoke(messages)
	state["critique"] = response.content
	
	# Determine if revision is needed (this logic might need refinement based on your specific requirements)
	state["needs_revision"] = "REVISION NEEDED" in response.content.upper()
	
	if state["needs_revision"]:
		state["revision_number"] += 1
	
	return state

In [63]:
from datetime import datetime


def finalization_node(state: AgentState) -> AgentState:
	# Here you can add any final processing, formatting, or metadata addition
	state["metadata"]["final_revision_count"] = state["revision_number"]
	state["metadata"]["finalization_timestamp"] = datetime.now().isoformat()
	
	# You might want to generate a final, formatted version of the summary here
	# For now, we'll just use the existing summary
	state["final_summary"] = state["summary"]
	
	return state

# Graph

In [80]:
def create_meeting_summary_workflow(checkpointer):
	workflow = StateGraph(AgentState)

	# Define nodes
	workflow.add_node("generate_summary_plan", summary_plan_node)
	workflow.add_node("generate_critique_plan", critique_plan_node)
	workflow.add_node("generate_summary", summarizer_node)
	workflow.add_node("perform_critique", critic_node)
	workflow.add_node("finalize_summary", finalization_node)

	# Define edges
	workflow.add_edge("generate_summary_plan", "generate_critique_plan")
	workflow.add_edge("generate_critique_plan", "generate_summary")
	workflow.add_edge("generate_summary", "perform_critique")
	workflow.add_conditional_edges(
		"perform_critique",
		lambda x: "generate_summary" if x["needs_revision"] and x["revision_number"] < x["max_revisions"] else "finalize_summary",
		{
			"generate_summary": "generate_summary",
			"finalize_summary": "finalize_summary"
		}
	)
	workflow.add_edge("finalize_summary", END)

	# Set entry point
	workflow.set_entry_point("generate_summary_plan")

	return workflow.compile(checkpointer = checkpointer)

In [61]:
workflow = StateGraph(AgentState)

# Define nodes with distinct names
workflow.add_node("generate_summary_plan", summary_plan_node)
workflow.add_node("generate_critique_plan", critique_plan_node)
workflow.add_node("generate_summary", summarizer_node)
workflow.add_node("perform_critique", critic_node)
workflow.add_node("finalize_summary", finalization_node)

# Define edges
workflow.add_edge("generate_summary_plan", "generate_critique_plan")
workflow.add_edge("generate_critique_plan", "generate_summary")
workflow.add_edge("perform_critique", "generate_summary")
workflow.add_conditional_edges(
	"perform_critique",
	lambda x: "generate_summary" if x["needs_revision"] and x["revision_number"] < x["max_revisions"] else "finalize_summary",
	{
		"generate_summary": "generate_summary",
		"finalize_summary": "finalize_summary"
	}
)
workflow.add_edge("finalize_summary", END)

# Set entry point
workflow.set_entry_point("generate_summary_plan")

In [81]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    graph = create_meeting_summary_workflow(memory)
    print(graph.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	generate_summary_plan(generate_summary_plan)
	generate_critique_plan(generate_critique_plan)
	generate_summary(generate_summary)
	perform_critique(perform_critique)
	finalize_summary(finalize_summary)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate_summary_plan;
	finalize_summary --> __end__;
	generate_critique_plan --> generate_summary;
	generate_summary --> perform_critique;
	generate_summary_plan --> generate_critique_plan;
	perform_critique -.-> generate_summary;
	perform_critique -.-> finalize_summary;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
transcript = load_markdown_to_str(file_path="../data/transcriptions/project-status-update-transcription.md")
context = [load_markdown_to_str(file_path="../data/company-data/it-company-profile.md"),load_markdown_to_str(file_path="../data/company-data/employee-profiles.md")]
initial_state = initialize_agent_state(
    meeting_type="Sprint planning meeting",
    transcript=transcript,
    main_topic=" To plan the 2 weeks of the sprint",
    context=context,
    max_revisions=2
)
print(initial_state)
print(type(initial_state))
pprint(initial_state)


In [ ]:
from pprint import pprint
from typing import List, Dict, Any

all_states: List[Dict[str, Any]] = []

with SqliteSaver.from_conn_string(":memory:") as memory:
    graph = create_meeting_summary_workflow(memory)
    thread = {"configurable": {"thread_id": "1"}}
    for s in graph.stream(initial_state, thread):
        pprint(s)
        all_states.append(s)

	
print(f"\nTotal number of states: {len(all_states)}")

# Access specific states
print("\nFirst state:")
pprint(all_states[0])

print("\nLast state:")
pprint(all_states[-1])

# You can also iterate through all states
for i, state in enumerate(all_states):
    print(f"\nState {i}:")
    pprint(state)

In [ ]:
# Access the last state
last_state = all_states[-1]

# Print the final summary from the last state
print("\nFinal Summary:")
print(last_state.get('final_summary'))
last_state.items().get
